In [80]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import pymysql
import csv
import pandas_profiling
import emoji
import pydotplus
import pydot
import scipy
import warnings
import matplotlib.pyplot
import sys
warnings.filterwarnings('ignore') 

from IPython.display import Image
from sqlalchemy import create_engine
from math import sqrt 
from numpy import int64
#SK Learn 
from sklearn.metrics import accuracy_score, classification_report,mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression 
from sklearn.neighbors import KNeighborsClassifier #KNN
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.datasets import load_digits
from sklearn.tree import plot_tree
from sklearn.tree import export_text
from sklearn import metrics
from sklearn import preprocessing
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection
from sklearn.metrics import confusion_matrix,precision_score,recall_score,accuracy_score,precision_recall_curve
from sklearn import tree
from sklearn.preprocessing import RobustScaler
%matplotlib inline

In [47]:
#df = pd.ExcelFile('Churn prediction data for web.xlsx')  
df = pd.read_excel('Churn prediction data for web (1).xlsx', engine='openpyxl')

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107514 entries, 0 to 107513
Data columns (total 22 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   UtilityAccountNumber  107514 non-null  object        
 1   AccountNumber         107514 non-null  object        
 2   PromoCode             4659 non-null    object        
 3   AttritionByMonth      97774 non-null   float64       
 4   EffectiveDate         103229 non-null  datetime64[ns]
 5   TerminationDate       47506 non-null   datetime64[ns]
 6   PremiseStatus         107514 non-null  object        
 7   PremiseType           107514 non-null  object        
 8   ServiceState          107511 non-null  object        
 9   ContractType          105740 non-null  object        
 10  GreenType             107514 non-null  int64         
 11  IsAutoPay             107514 non-null  int64         
 12  Commodity             107514 non-null  object        
 13 

In [49]:
df.head()

,UtilityAccountNumber,AccountNumber,PromoCode,AttritionByMonth,EffectiveDate,TerminationDate,PremiseStatus,PremiseType,ServiceState,ContractType,...,Commodity,ProductType,ProductTerm,mastercompany,IsEBill,Rate,AttritionType,SoldDate,activebymonth,Churn
0,035013237595,002752814,NaN,63.0,2015-01-08,2020-04-06,INACTIVE,RESIDENTIAL,ME,RENEWAL,...,ELECTRIC,Fixed,12.0,PROVIDER,0,0.0899,FAILURE TO RENEW,2014-12-08,64,1
1,2102956001,002789231,NaN,0.0,2015-03-20,NaT,ACTIVE,RESIDENTIAL,NH,RENEWAL,...,ELECTRIC,Fixed,12.0,PROVIDER,0,0.1307,NaN,2015-03-09,71,0
2,10042539,002750405,NaN,0.0,2012-07-11,NaT,ACTIVE,RESIDENTIAL,ME,RENEWAL,...,ELECTRIC,Fixed,1.0,PROVIDER,0,0.1299,NaN,2013-09-27,113,0
3,035014215061,002756528,NaN,0.0,2012-04-07,NaT,ACTIVE,RESIDENTIAL,ME,RENEWAL,...,ELECTRIC,Fixed,1.0,PROVIDER,0,0.1030,NaN,2013-09-27,111,0
4,035016437226,002774599,NaN,85.0,2012-07-07,2019-08-06,INACTIVE,RESIDENTIAL,ME,RENEWAL,...,ELECTRIC,Fixed,1.0,PROVIDER,0,0.1325,FAILURE TO RENEW,2013-09-27,86,1


In [50]:
df.iloc[1]

UtilityAccountNumber             2102956001
AccountNumber                     002789231
PromoCode                               NaN
AttritionByMonth                          0
EffectiveDate           2015-03-20 00:00:00
TerminationDate                         NaT
PremiseStatus                        ACTIVE
PremiseType                     RESIDENTIAL
ServiceState                             NH
ContractType                        RENEWAL
GreenType                                 0
IsAutoPay                                 0
Commodity                          ELECTRIC
ProductType                           Fixed
ProductTerm                              12
mastercompany                      PROVIDER
IsEBill                                   0
Rate                                 0.1307
AttritionType                           NaN
SoldDate                2015-03-09 00:00:00
activebymonth                            71
Churn                                     0
Name: 1, dtype: object

In [51]:
#To find if your data set has null 
print(df.isnull().sum())

UtilityAccountNumber         0
AccountNumber                0
PromoCode               102855
AttritionByMonth          9740
EffectiveDate             4285
TerminationDate          60008
PremiseStatus                0
PremiseType                  0
ServiceState                 3
ContractType              1774
GreenType                    0
IsAutoPay                    0
Commodity                    0
ProductType               2197
ProductTerm               2197
mastercompany                0
IsEBill                      0
Rate                      6186
AttritionType            65894
SoldDate                    12
activebymonth                0
Churn                        0
dtype: int64


In [52]:
df.UtilityAccountNumber.duplicated().sum()

2778

In [53]:
df.AccountNumber.duplicated().sum()

151

In [54]:
df.AccountNumber.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
107509    False
107510    False
107511    False
107512    False
107513    False
Name: AccountNumber, Length: 107514, dtype: bool

In [55]:
df.duplicated().sum()

2

In [56]:
#to look at the duplicated rows 
df.loc[df.duplicated(),:]

,UtilityAccountNumber,AccountNumber,PromoCode,AttritionByMonth,EffectiveDate,TerminationDate,PremiseStatus,PremiseType,ServiceState,ContractType,...,Commodity,ProductType,ProductTerm,mastercompany,IsEBill,Rate,AttritionType,SoldDate,activebymonth,Churn
95993,10443720002763199,004198974,NaN,NaN,2019-12-23,2019-12-23,ENROLLMENT CANCELLED,RESIDENTIAL,TX,ENROLLMENT,...,ELECTRIC,Fixed,6.0,SPARK,1,0.048,NaN,2019-12-19,6,1
100064,10443720004556924,004198962,NAVC-KIOSK_104,NaN,2019-12-18,2019-12-18,ENROLLMENT CANCELLED,RESIDENTIAL,TX,ENROLLMENT,...,ELECTRIC,Fixed,36.0,SPARK,0,0.089,NaN,2019-12-18,36,1


In [57]:
#drop all the duplicates
df = df.drop_duplicates()

In [58]:
df.duplicated().sum()

0

In [59]:
df = df[df['ContractType'].notna()]

In [60]:
df = df[df['Rate'].notna()]

In [61]:
df = df[df['ServiceState'].notna()]

In [62]:
#To find if your data set has null 
print(df.isnull().sum())

UtilityAccountNumber        0
AccountNumber               0
PromoCode               96258
AttritionByMonth         4498
EffectiveDate             165
TerminationDate         55857
PremiseStatus               0
PremiseType                 0
ServiceState                0
ContractType                0
GreenType                   0
IsAutoPay                   0
Commodity                   0
ProductType                 0
ProductTerm                 0
mastercompany               0
IsEBill                     0
Rate                        0
AttritionType           60320
SoldDate                    0
activebymonth               0
Churn                       0
dtype: int64


In [63]:

#Removed the dup column 
df = df.drop(['AttritionType','PromoCode'],axis=1)

In [64]:
df = df[df['EffectiveDate'].notna()]

In [65]:
#To find if your data set has null 
print(df.isnull().sum())

UtilityAccountNumber        0
AccountNumber               0
AttritionByMonth         4333
EffectiveDate               0
TerminationDate         55692
PremiseStatus               0
PremiseType                 0
ServiceState                0
ContractType                0
GreenType                   0
IsAutoPay                   0
Commodity                   0
ProductType                 0
ProductTerm                 0
mastercompany               0
IsEBill                     0
Rate                        0
SoldDate                    0
activebymonth               0
Churn                       0
dtype: int64


In [66]:
 df['AttritionByMonth'].fillna(0)


0         63.0
1          0.0
2          0.0
3          0.0
4         85.0
          ... 
107509     0.0
107510     0.0
107511    20.0
107512     0.0
107513     0.0
Name: AttritionByMonth, Length: 99887, dtype: float64

In [67]:
df['AttritionByMonth'] = df['AttritionByMonth'].fillna(0)

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99887 entries, 0 to 107513
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   UtilityAccountNumber  99887 non-null  object        
 1   AccountNumber         99887 non-null  object        
 2   AttritionByMonth      99887 non-null  float64       
 3   EffectiveDate         99887 non-null  datetime64[ns]
 4   TerminationDate       44195 non-null  datetime64[ns]
 5   PremiseStatus         99887 non-null  object        
 6   PremiseType           99887 non-null  object        
 7   ServiceState          99887 non-null  object        
 8   ContractType          99887 non-null  object        
 9   GreenType             99887 non-null  int64         
 10  IsAutoPay             99887 non-null  int64         
 11  Commodity             99887 non-null  object        
 12  ProductType           99887 non-null  object        
 13  ProductTerm    

In [69]:
df.head()

,UtilityAccountNumber,AccountNumber,AttritionByMonth,EffectiveDate,TerminationDate,PremiseStatus,PremiseType,ServiceState,ContractType,GreenType,IsAutoPay,Commodity,ProductType,ProductTerm,mastercompany,IsEBill,Rate,SoldDate,activebymonth,Churn
0,035013237595,002752814,63.0,2015-01-08,2020-04-06,INACTIVE,RESIDENTIAL,ME,RENEWAL,0,0,ELECTRIC,Fixed,12.0,PROVIDER,0,0.0899,2014-12-08,64,1
1,2102956001,002789231,0.0,2015-03-20,NaT,ACTIVE,RESIDENTIAL,NH,RENEWAL,0,0,ELECTRIC,Fixed,12.0,PROVIDER,0,0.1307,2015-03-09,71,0
2,10042539,002750405,0.0,2012-07-11,NaT,ACTIVE,RESIDENTIAL,ME,RENEWAL,0,0,ELECTRIC,Fixed,1.0,PROVIDER,0,0.1299,2013-09-27,113,0
3,035014215061,002756528,0.0,2012-04-07,NaT,ACTIVE,RESIDENTIAL,ME,RENEWAL,0,0,ELECTRIC,Fixed,1.0,PROVIDER,0,0.1030,2013-09-27,111,0
4,035016437226,002774599,85.0,2012-07-07,2019-08-06,INACTIVE,RESIDENTIAL,ME,RENEWAL,0,0,ELECTRIC,Fixed,1.0,PROVIDER,0,0.1325,2013-09-27,86,1


In [84]:
#pd.to_numeric(df['UtilityAccountNumber'], errors='coerce',downcast='signed')

#pd.to_numeric(df['AccountNumber'], errors='coerce',downcast='integer')

0         2752814.0
1         2789231.0
2         2750405.0
3         2756528.0
4         2774599.0
            ...    
107509    4307511.0
107510    4306815.0
107511    4163901.0
107512    4306118.0
107513    4305653.0
Name: AccountNumber, Length: 99887, dtype: float64

In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99887 entries, 0 to 107513
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   UtilityAccountNumber  99887 non-null  object        
 1   AccountNumber         99887 non-null  object        
 2   AttritionByMonth      99887 non-null  float64       
 3   EffectiveDate         99887 non-null  datetime64[ns]
 4   TerminationDate       44195 non-null  datetime64[ns]
 5   PremiseStatus         99887 non-null  object        
 6   PremiseType           99887 non-null  object        
 7   ServiceState          99887 non-null  object        
 8   ContractType          99887 non-null  object        
 9   GreenType             99887 non-null  int64         
 10  IsAutoPay             99887 non-null  int64         
 11  Commodity             99887 non-null  object        
 12  ProductType           99887 non-null  object        
 13  ProductTerm    